In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("Downloading dataset...")
!wget -q https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

zip_ref = zipfile.ZipFile("10_food_classes_all_data.zip", "r")
zip_ref.extractall()
zip_ref.close()

print("Dataset downloaded!")

train_dir = "10_food_classes_all_data/train/"
test_dir = "10_food_classes_all_data/test/"

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

print("\nLoading images...")
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    classes=['pizza', 'hamburger', 'ice_cream']
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    classes=['pizza', 'hamburger', 'ice_cream']
)

calories_dict = {
    'pizza': '266 kcal (1 slice)',
    'hamburger': '295 kcal (1 burger)',
    'ice_cream': '207 kcal (1 scoop)'
}

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print("\nStarting Training (approx 2-3 mins)...")
history = model.fit(train_data, epochs=5, validation_data=test_data)

print("Training Complete!")

def predict_food(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)

    labels = list(train_data.class_indices.keys())
    predicted_label = labels[class_index]

    cal_info = calories_dict[predicted_label]

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Pred: {predicted_label.upper()}\nEst. Energy: {cal_info}", color='green')
    plt.show()


import random
sample_dir = "10_food_classes_all_data/test/pizza/"
sample_file = random.choice(os.listdir(sample_dir))
predict_food(sample_dir + sample_file)